In [41]:
#Inicializacion de spark session y creacion de nuestra instancia
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType,IntegerType,StringType,FloatType

spark = SparkSession.builder.appName('Sesion3_nulos').getOrCreate()

In [42]:
ventas = spark.read.csv('VentasNulos.csv',inferSchema=True,header=True)

In [43]:
ventas.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Ventas: integer (nullable = true)
 |-- Euros: integer (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Identificador: string (nullable = true)



In [44]:
ventas.show()
#Vemos que tenemos algunos datos nulos

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
|  Juan|  null| null|    null|          C54|
| Pedro|     1|   30|Valencia|          R23|
| Maria|  null|  300|  Murcia|         null|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
|  null|  null| null|    null|         null|
| Jorge|     8| null|    null|          T19|
+------+------+-----+--------+-------------+



# BORRADO DE NULOS

# na.drop()
Con esto vamos a poder borrar todas las filas que tengan algun nulo

In [45]:
ventas.na.drop().show()
#Como vemos nos quedamos solo con las filas que tienen todos los campos informados

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
| Pedro|     1|   30|Valencia|          R23|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
+------+------+-----+--------+-------------+



# na.drop(thresh = X)
Usando esta funcion podremos elegir el numero de campos que queremos que esten NO sean null para descartar el resto

In [46]:
ventas.na.drop(thresh = 3).show()
#Como hemos elegido el thresh = 3 nos va a quitar de la lista las filas que tengan al menos 3 datos no nulos, 
#Como por ejemllo Juan 
#Juan|  null| null|    null|          C54  
#solo tiene 2 datos no nulos, entonces lo quitamos

#Es decir de 5 columnas que tenemos le estamos diciendo que minimo queremos que 3 esten informadas, solemos pensar que el valor
#que le pasamos es el numero de nulos, pero no, es al reves, es el numero de NO NULOS

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
| Pedro|     1|   30|Valencia|          R23|
| Maria|  null|  300|  Murcia|         null|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
| Jorge|     8| null|    null|          T19|
+------+------+-----+--------+-------------+



In [47]:
#Si ponemos a 5, estaremos indicando que no queremos que hayan nulos y solo nos devolvera las fulas que no tienen ningun nulo
ventas.na.drop(thresh = 5).show()

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
| Pedro|     1|   30|Valencia|          R23|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
+------+------+-----+--------+-------------+



In [48]:
#Simplemente entender que el valor que le pasamos es de los NO NULOS que queremos que hayan

# na.drop(how=any)
En este caso podremos indicar how = any para que nos borre todas las que contengan nulos, seria equivalente al thresh 5 que 
hemos visto justo antes

In [49]:
ventas.na.drop(how = 'any').show()

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
| Pedro|     1|   30|Valencia|          R23|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
+------+------+-----+--------+-------------+



# na.drop(how='all')
Usando how a all, podremos eliminar las filas que todos sus campos sean nulos

In [50]:
ventas.na.drop(how = 'all').show()

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
|  Juan|  null| null|    null|          C54|
| Pedro|     1|   30|Valencia|          R23|
| Maria|  null|  300|  Murcia|         null|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
| Jorge|     8| null|    null|          T19|
+------+------+-----+--------+-------------+



# na.drop(subset=COLUMNA)
De este modo podremos eliminar la fila que en la columna indicada haya algun nulo, por ejemplo no nos interesan los 
registros que tengan la columna Euros a nula

In [51]:
ventas.na.drop(subset = 'Euros').show()

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
| Pedro|     1|   30|Valencia|          R23|
| Maria|  null|  300|  Murcia|         null|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
+------+------+-----+--------+-------------+



# REEMPLAZO DE NULOS

In [52]:
#Ahora veremos como podemos en vez de eliminar la fila entera que tenga nulos, como podemos darle valor a estos nulos, y 
#asi no perder el resto de registros de la fila

# fill()

In [53]:
ventas.na.fill(0).show()
#Como vemos nos rellena con 0, pero obviamente solo los campos que son numericos, el resto siguien estando a null

+------+------+-----+--------+-------------+
|Nombre|Ventas|Euros|  Ciudad|Identificador|
+------+------+-----+--------+-------------+
|  Pepe|     4|  200|   Elche|          X21|
|  Juan|     0|    0|    null|          C54|
| Pedro|     1|   30|Valencia|          R23|
| Maria|     0|  300|  Murcia|         null|
|  Rosa|     3|  350|  Murcia|          V55|
|   Ana|    10| 2300|Alicante|          B89|
|  null|     0|    0|    null|         null|
| Jorge|     8|    0|    null|          T19|
+------+------+-----+--------+-------------+



In [62]:
#Por el contrario, si hacemos un reemplazo por un string ocurrira lo mismo, los numericos se quedan con null:
ventas.na.fill("Desconocido").show()

+-----------+------+-----+-----------+-------------+
|     Nombre|Ventas|Euros|     Ciudad|Identificador|
+-----------+------+-----+-----------+-------------+
|       Pepe|     4|  200|      Elche|          X21|
|       Juan|  null| null|Desconocido|          C54|
|      Pedro|     1|   30|   Valencia|          R23|
|      Maria|  null|  300|     Murcia|  Desconocido|
|       Rosa|     3|  350|     Murcia|          V55|
|        Ana|    10| 2300|   Alicante|          B89|
|Desconocido|  null| null|Desconocido|  Desconocido|
|      Jorge|     8| null|Desconocido|          T19|
+-----------+------+-----+-----------+-------------+



In [64]:
#Podemos pasarle dos intercambios de nulos, en string y en string, y todos los que sean string o numerico, se cambian por
#lo que le digamos

ventas.na.fill("Desconocido").na.fill(0).show()

+-----------+------+-----+-----------+-------------+
|     Nombre|Ventas|Euros|     Ciudad|Identificador|
+-----------+------+-----+-----------+-------------+
|       Pepe|     4|  200|      Elche|          X21|
|       Juan|     0|    0|Desconocido|          C54|
|      Pedro|     1|   30|   Valencia|          R23|
|      Maria|     0|  300|     Murcia|  Desconocido|
|       Rosa|     3|  350|     Murcia|          V55|
|        Ana|    10| 2300|   Alicante|          B89|
|Desconocido|     0|    0|Desconocido|  Desconocido|
|      Jorge|     8|    0|Desconocido|          T19|
+-----------+------+-----+-----------+-------------+



In [ ]:
#Podemos también usar intercambios por columnas por ejemplo: a la columna ciudad queremos ponerle el string Desconocido 
#y a la Identificadorr XXX y si hay numericos nulos, le ponemos 0

In [61]:
ventas.na.fill("Desconocido",["Ciudad"]).na.fill("XXX",["Identificador"]).na.fill(0).show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|  Juan|     0|    0|Desconocido|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| Maria|     0|  300|     Murcia|          XXX|
|  Rosa|     3|  350|     Murcia|          V55|
|   Ana|    10| 2300|   Alicante|          B89|
|  null|     0|    0|Desconocido|          XXX|
| Jorge|     8|    0|Desconocido|          T19|
+------+------+-----+-----------+-------------+



# fillna()
Tenemos otra forma de hacer estos intercambios y es con fillna, es similar pero el formato cambia y podemos hacer todo
de golpe con una sola llamada, vamos a ver como funciona:


In [71]:

ventas.fillna({"Nombre": "Desconocido", "Ciudad": "De España"}).show()

+-----------+------+-----+---------+-------------+
|     Nombre|Ventas|Euros|   Ciudad|Identificador|
+-----------+------+-----+---------+-------------+
|       Pepe|     4|  200|    Elche|          X21|
|       Juan|  null| null|De España|          C54|
|      Pedro|     1|   30| Valencia|          R23|
|      Maria|  null|  300|   Murcia|         null|
|       Rosa|     3|  350|   Murcia|          V55|
|        Ana|    10| 2300| Alicante|          B89|
|Desconocido|  null| null|De España|         null|
|      Jorge|     8| null|De España|          T19|
+-----------+------+-----+---------+-------------+

